In [1]:
import sys
sys.path.append("d:\\OneDrive - Universiteit Utrecht\\Documents\\000 - Documenti\\PROJECTS\\x-tree-search")

In [2]:
from games.tic_tac_toe import TicTacToe
from games.game import GameAgent, User
from algorithms.minmax import MinMax
from explainers.minmax_explainer import MinMaxExplainer

# Define game

In [3]:
game = TicTacToe()

# Define tic-tac-toe scoring function for the minmax

In [4]:
import numpy as np
def scoring_function(node):
    """ Evaluate the Tic Tac Toe board state for the 'X' player's perspective """
    state = node.state
    score = 0
    
    # Possible lines to check (3 rows, 3 columns, 2 diagonals)
    lines = []
    # Rows and columns
    for i in range(3):
        lines.append(state[i, :])  # Row
        lines.append(state[:, i])  # Column
    # Diagonals
    lines.append(np.array([state[i, i] for i in range(3)]))  # Main diagonal
    lines.append(np.array([state[i, 2 - i] for i in range(3)]))  # Anti-diagonal

    for line in lines:
        if np.all(line == "X"):
            score += 100  # 'X' wins
        elif np.all(line == "O"):
            score -= 100  # 'O' wins
        elif np.count_nonzero(line == "X") == 2 and np.count_nonzero(line == "free") == 1:
            score += 10  # 'X' is one move away from winning
        elif np.count_nonzero(line == "O") == 2 and np.count_nonzero(line == "free") == 1:
            score -= 10  # 'O' is one move away from winning

    return score

# Define explainer

In [5]:
explainer = MinMaxExplainer()

# Play and Explain

In [6]:
opponent = GameAgent(MinMax(scoring_function), agent_id=0)
user = User(agent_id=1, ask_what = False)

In [7]:
game.set_players([opponent, user])

In [11]:
await game.start_on_jupyter()

[['O' 'X' ' ']
 [' ' 'X' ' ']
 [' ' ' ' 'O']]

Game paused.

[['O' 'X' ' ']
 [' ' 'X' ' ']
 [' ' ' ' 'O']]


In [12]:
choice = opponent.core.last_choice
explainer.explain_adjective(choice, 'best')

0_2 is best ←
 	(assumption) By definition a node is "best" if it's "better" than all "siblings"
	∧ 0_2 has siblings = 0_0, 0_1, 0_3, 0_4, 0_5 ←
	 		(assumption) Definition of "siblings" is node.parent.children excluding node
	∧ 0_2 is better than 0_0 ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score >= node2 score
			∧ 0_2 has score = 0 ←
			 			0_2 is ¬(leaf)
						∧ (assumption) Internal nodes have scores from children
						∧ 0_2 has backpropagating child = 0_2_4
			∧ 0_0 has score = -100 ←
			 			0_0 is ¬(leaf)
						∧ (assumption) Internal nodes have scores from children
						∧ 0_0 has backpropagating child = 0_0_1
	∧ 0_2 is better than 0_1 ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score >= node2 score
			∧ 0_2 has score = 0 ←
			 			0_2 is ¬(leaf)
						∧ (assumption) Internal nodes have scores from children
						∧ 0_2 has backpropagating child = 0_2_4
			∧ 0_1 has score = -100 ←
			 			0_1 is ¬(leaf)
						∧ (assumptio